# Algorytm k najbliższych sąsiadów

### Model

Model najbliższych sąsiadów to najprostrzy istniejący model predyktywny. Wymago on jedynie:
- miar odległości
- założenia, że punkty znajdujące się bliżej siebie są do siebie bardziej podobne

Załóżmy, że wybraliśmy wartość k równą np. 3 lub 5 i chcemy dokonać klasyfikacji nowych obserwacji - znaleźć k najbliższych elementów zbioru danych, oznaczonych etykietami i na ich podstawie ustalić etykietę nowej obserwacji.

W tym celu potrzebujemy funkcji liczącej głosy.

In [1]:
from typing import List
from collections import Counter

def raw_majority_vote(labels: List[str]) -> str:
    votes = Counter(labels)
    winner, _ = votes.most_common(1)[0]
    return winner

assert raw_majority_vote(['a', 'b', 'c', 'b']) == 'b'

Załóżmy, że określamy kategorię wiekową filmów i pięciu najbliższym filmom nadano kategorie G, G, PG, PG i R. Mamy więc dwa głosy na kategorię G i dwa głosy na PG. W takim przypadku możemy skorzystać z kilku rozwiązań:
- wybrać zwycięzcę w sposób losowy
- określić wagi głosów na podstawie odległości i wybra zwycięzcę na podstawie wagi
- zmniejszać wartość parametru k aż do jednoznacznego określenia zwycięzcy.

Trzecie rozwiązanie:

In [2]:
def majority_vote(labels: List[str]) -> str:
    """Funkcja zakłada, że etykiety są ustawione w kolejności od najbliższej do najdalszej."""
    vote_counts = Counter(labels)
    winner, winner_count = vote_counts.most_common(1)[0]
    num_winners = len([count
                       for count in vote_counts.values()
                       if count == winner_count])

    if num_winners == 1:
        return winner                     # Ustalono jednoznacznie zwycięzcę. Zwróć go.
    else:
        return majority_vote(labels[:-1]) # Odrzuć najdalszą obserwację i spróbuj ponownie.

majority_vote(['a', 'b', 'c', 'b', 'a'])

'b'

In [3]:
# Remis, więc patrzymy na pierwsze 4 i powinno wyjść 'b'
assert majority_vote(['a', 'b', 'c', 'b', 'a']) == 'b'

In [4]:
# klasyfikator

from typing import List

Vector = List[float]

# Moduł wektora
import math

def magnitude(v: Vector) -> float:
    """Zwraca moduł (długość) wektora v"""
    return math.sqrt(sum_of_squares(v))   # Funkcja math.sqrt oblicza wartość pierwiastka kwadratowego.

# Odległości pomiędzy dwoma wektorami
def squared_distance(v: Vector, w: Vector) -> float:
    """Oblicza (v_1 - w_1) ** 2 + ... + (v_n - w_n) ** 2"""
    return sum_of_squares(subtract(v, w))

def distance(v: Vector, w: Vector) -> float:
    """Oblicza odległość pomiędzy v i w"""
    return math.sqrt(squared_distance(v, w))

def distance(v: Vector, w: Vector) -> float:  # type: ignore
    return magnitude(subtract(v, w))

In [5]:
from typing import NamedTuple
class LabeledPoint(NamedTuple):
    point: Vector
    label: str

def knn_classify(k: int,
                 labeled_points: List[LabeledPoint],
                 new_point: Vector) -> str:

    # Ustaw punkty oznaczone etykietami w kolejności od najbliższego do najdalszego.
    by_distance = sorted(labeled_points,
                         key=lambda lp: distance(lp.point, new_point))

    # Ustal etykiety k najbliższych punktów.
    k_nearest_labels = [lp.label for lp in by_distance[:k]]

    # Wybierz zwycięzcę na podstawie tych etykiet.
    return majority_vote(k_nearest_labels)